# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [6]:
! hadoop fs -copyFromLocal ../clickstream.csv

copyFromLocal: `clickstream.csv': File exists


In [83]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
se = SparkSession(sc)

2024-10-25 17:01:47,349 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [84]:
# Load the clickstream data with proper schema and types
clickstream_df = se.read.option("delimiter", "\t") \
    .option("header", True) \
    .csv('clickstream.csv')

## Spark SQL

In [85]:
# Cast timestamp to integer for processing
clickstream_df = clickstream_df.withColumn("timestamp", clickstream_df.timestamp.cast("int"))

In [86]:
# Identify minimum timestamp for sessions containing errors
min_timestamp_df = (clickstream_df
    .filter(F.col("event_type").contains("error"))  # filter for error events
    .groupBy("user_id", "session_id")
    .agg(F.min("timestamp").alias("min_ts"))  # calculate the minimum timestamp
)

In [87]:
# Define a window specification to track previous event pages
window_spec = Window.partitionBy("user_id", "session_id").orderBy("timestamp")

In [88]:
# Process the DataFrame to extract valid routes
valid_routes_df = (clickstream_df
    .join(min_timestamp_df, ["user_id", "session_id"], "left")  # perform a left join
    .filter(
        (F.col("min_ts").isNull()) | (F.col("timestamp") < F.col("min_ts"))
    )  # keep valid sessions
    .filter(F.col("event_type") == "page")  # keep only page events
    .select("user_id", "session_id", "event_page", "timestamp")  # select necessary columns
    .withColumn("prev_event_page", F.lag("event_page", 1).over(window_spec))  # get previous page
    .filter(
        (F.col("prev_event_page").isNull()) | (F.col("event_page") != F.col("prev_event_page"))
    )  # remove consecutive duplicates
)

In [89]:
# Aggregate pages into routes and count occurrences
routes_df = (valid_routes_df
    .groupBy("user_id", "session_id")
    .agg(F.collect_list("event_page").alias("pages"))  # collect pages into a list
    .withColumn("route", F.array_join("pages", "-"))  # create route string
    .groupBy("route")
    .agg(F.count("*").alias("count"))  # count occurrences of each route
    .orderBy(F.desc("count"))  # sort by count in descending order
)

In [90]:
# The 30 most frequent user routes
routes_df.show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1113|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  676|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|    main-news-rabota|  135|
|   main-bonus-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
| main-internet-bonus|  115|
|main-tariffs-inte...|  114|
|   main-news-archive|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|  main-internet-news|  103|
|main-tariffs-archive|  103|
|    main-rabota-main|   94|
+--------------------+-----+
only showing t

## Spark DF

In [92]:
# Process the DataFrame to extract valid routes
result_df = (clickstream_df
    .join(min_timestamp_df, ['user_id', 'session_id'], 'left')  # join with min_timestamp_df
    .filter((min_timestamp_df.min_ts.isNull()) | (clickstream_df.timestamp < min_timestamp_df.min_ts))  # filter out invalid sessions
    .filter(clickstream_df.event_type == 'page')  # keep only "page" events
    .select('user_id', 'session_id', 'event_page', 'timestamp')  # select relevant columns
    .withColumn('prev_event_page', F.lag('event_page', 1).over(window_spec))  # get the previous event page
    .filter((F.col('prev_event_page').isNull()) | (F.col('event_page') != F.col('prev_event_page')))  # remove consecutive duplicates
    .groupBy('user_id', 'session_id')  # group by user and session
    .agg(F.collect_list('event_page').alias('pages'))  # collect pages in a list
    .withColumn('route', F.array_join('pages', '-'))  # join pages into a route string
    .groupBy('route')  # group by route
    .agg(F.count('*').alias('count'))  # count occurrences of each route
    .orderBy(F.desc('count'))  # sort by count in descending order
)

In [93]:
# The 30 most frequent user routes
result_df.show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1113|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  676|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|    main-news-rabota|  135|
|   main-bonus-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
| main-internet-bonus|  115|
|main-tariffs-inte...|  114|
|   main-news-archive|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|  main-internet-news|  103|
|main-tariffs-archive|  103|
|    main-rabota-main|   94|
+--------------------+-----+
only showing t

## Spark RDD

In [94]:
clickstream_rdd = clickstream_df.rdd

In [95]:
# Identify sessions with errors and the corresponding timestamps
error_events = (clickstream_rdd
    .filter(lambda line: 'error' in line['event_type'])
    .map(lambda line: (f"{line['user_id']}_{line['session_id']}", int(line['timestamp'])))
    .groupByKey()
    .map(lambda line:(line[0], min(line[1])))  # find the earliest timestamp of the error
    .collectAsMap())

In [96]:
# Broadcast the error events map to optimize for large datasets
errors_broadcast = sc.broadcast(error_events)

In [97]:
# Process the clickstream data to filter valid sessions and construct routes
import math
from itertools import groupby


def remove_consecutive_duplicates(obj):
    sorted_pages = [i[0] for i in sorted(obj[1], key=lambda o: o[1])]  # sort pages by their timestamp
    pages = [key for key, _ in groupby(sorted_pages)]  # remove consecutive duplicates
    return (obj[0], pages)

In [98]:
filtered_routes = (clickstream_rdd
    .map(lambda line: (f"{line['user_id']}_{line['session_id']}", [line['event_type'], line['event_page'], int(line['timestamp'])]))
    .filter(lambda line: line[0] not in errors_broadcast.value or line[1][2] < errors_broadcast.value[line[0]])
    .filter(lambda line: line[1][0] == "page")  # only keep "page" events
    .map(lambda line: (line[0], (line[1][1], line[1][2])))  # extract relevant info
    .groupByKey()
    .map(remove_consecutive_duplicates)  # remove consecutive duplicates
    .map(lambda line: (line[0], "-".join(line[1])))  # join pages into routes
    .map(lambda line: (line[1], 1))  # map routes to counts
    .reduceByKey(lambda x, y: x + y)  # sum counts for each route
    .sortBy(lambda line: line[1], ascending=False)  # sort routes by count in descending order
    .take(30)  # take the top 30 routes
)

In [99]:
# The 30 most frequent user routes
filtered_routes

[('main', 8184),
 ('main-archive', 1113),
 ('main-rabota', 1047),
 ('main-internet', 897),
 ('main-bonus', 870),
 ('main-news', 769),
 ('main-tariffs', 677),
 ('main-online', 587),
 ('main-vklad', 518),
 ('main-rabota-archive', 170),
 ('main-archive-rabota', 167),
 ('main-bonus-archive', 143),
 ('main-rabota-bonus', 139),
 ('main-news-rabota', 135),
 ('main-bonus-rabota', 135),
 ('main-archive-internet', 132),
 ('main-rabota-news', 130),
 ('main-internet-rabota', 129),
 ('main-archive-news', 126),
 ('main-rabota-internet', 124),
 ('main-internet-archive', 123),
 ('main-archive-bonus', 117),
 ('main-internet-bonus', 115),
 ('main-tariffs-internet', 114),
 ('main-news-archive', 113),
 ('main-news-internet', 109),
 ('main-archive-tariffs', 104),
 ('main-tariffs-archive', 103),
 ('main-internet-news', 103),
 ('main-rabota-main', 94)]

## Submission

In [100]:
top_10_routes = {}
for k, v in filtered_routes[:10]:
    top_10_routes[k] = v

In [101]:
top_10_routes

{'main': 8184,
 'main-archive': 1113,
 'main-rabota': 1047,
 'main-internet': 897,
 'main-bonus': 870,
 'main-news': 769,
 'main-tariffs': 677,
 'main-online': 587,
 'main-vklad': 518,
 'main-rabota-archive': 170}

In [102]:
import json

with open("result.json", "w") as f:
    json.dump(top_10_routes, f)

In [103]:
!curl -F file=@result.json 51.250.123.136:80/MDS-LSML1/jenyatsvetkov/w6/2

0.9999999999999999
Correct main answer!
Correct main-archive answer!
Correct main-rabota answer!
Correct main-internet answer!
Correct main-bonus answer!
Correct main-news answer!
Correct main-tariffs answer!
Correct main-online answer!
Correct main-vklad answer!
Correct main-rabota-archive answer!


In [104]:
sc.stop()